In [2]:
import cv2
import numpy as np

# Load the image
folder = 'c:\\Users\\proje\\Documents\\GitHub\\Bowser-attack\\bowser\\base_segment\\test_screenshots\\'
image = cv2.imread(folder + 'Screenshot 2023-11-21 151349.png')

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Thresholding
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Noise removal
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

# Sure background area
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers + 1

# Now, mark the region of unknown with zero
markers[unknown == 255] = 0

# Apply Watershed
markers = cv2.watershed(image, markers)
image[markers == -1] = [0, 255, 0]

# Display the result
cv2.imshow('Segmented Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [31]:
import cv2
import pytesseract
import random
import numpy as np
import math

# Configure Tesseract path

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


# Load the image
folder = 'c:\\Users\\proje\\Documents\\GitHub\\Bowser-attack\\bowser\\base_segment\\test_screenshots\\'
image = cv2.imread(folder + 'Screenshot 2023-11-21 134221.png')

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

# Iterate through OCR result and remove text
n_boxes = len(data['level'])
for i in range(n_boxes):
    if int(data['conf'][i]) > 60:  # Confidence threshold
        (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), -1)
        print(w, h)

def random_color():
    """ Generate random bright colors """
    return (random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))

# Load the image

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)



# Apply Canny Edge Detection
edges = cv2.Canny(gray_image, 100, 200)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Create an all zeros mask with the same dimensions as the input image
mask = np.zeros_like(image)

# Draw contours and fill the area with a semi-transparent color
for cnt in contours:
    color = random_color()
    cv2.drawContours(mask, [cnt], 0, color, -1)  # Fill contour
    cv2.drawContours(mask, [cnt], 0, color, 1)  # Draw contour line

# Blend the mask with the image
alpha = 0.7  # Transparency factor
segmented_image = cv2.addWeighted(image, alpha, mask, 1 - alpha, 0)

for i in range(n_boxes):
    if int(data['conf'][i]) > 0:  # Confidence threshold to filter weak detections
        (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
        image = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red rectangle
        text = data['text'][i]
        cv2.putText(segmented_image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(segmented_image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

# Display the result
cv2.imshow('Segmented Image with Highlights', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


64 39
83 18
12 18
589 189
78 15
62 15
10 15


In [49]:
import cv2
import easyocr
import numpy as np
import random

def random_color():
    """ Generate random bright colors """
    return (random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))

def process_image(image_path):
    # Initialize EasyOCR reader
    reader = easyocr.Reader(['en'])

    # Load the image

    folder = 'c:\\Users\\proje\\Documents\\GitHub\\Bowser-attack\\bowser\\base_segment\\test_screenshots\\'
    image = cv2.imread(folder + 'Screenshot 2023-11-21 134324.png')
    original_size = image.shape[:2]

    masked_image = image.copy()
    # Run EasyOCR on the image
    results = reader.readtext(image)

    # Process each detected text box
    for (bbox, text, prob) in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = (int(top_left[0]), int(top_left[1]))
        bottom_right = (int(bottom_right[0]), int(bottom_right[1]))

        # Check if the bounding box is smaller than 75 pixels in one dimension
        if (bottom_right[0] - top_left[0]) < 75 or (bottom_right[1] - top_left[1]) < 75:
            cv2.rectangle(masked_image, top_left, bottom_right, (255, 255, 255), -1)

        # Draw the text box and text on the original image
        cv2.rectangle(image, top_left, bottom_right, (0, 0, 255), 2)
        text_loc = (top_left[0], top_left[1] - 10)
        cv2.putText(image, text, text_loc, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(image, text, text_loc, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Convert to grayscale and apply Canny Edge Detection on masked image
    # Convert to grayscale
    gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)

    # Noise reduction with Gaussian Blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Contrast enhancement with Histogram Equalization
    contrast_enhanced = cv2.equalizeHist(blurred)

    # Apply Canny Edge Detection on contrast-enhanced image
    edges = cv2.Canny(contrast_enhanced, 100,200)

    
    #sobelx = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)  # Sobel Edge Detection on the X axis
    #sobely = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)  # Sobel Edge Detection on the Y axis
    #sobel = np.sqrt(sobelx ** 2 + sobely ** 2)  # Combine X and Y Sobel results

    #laplacian = cv2.Laplacian(blurred, cv2.CV_64F)

    # Find contours from edges
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


    # List to store individual contour images
    contour_images = []

    square_size = 300

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 30:
            color = (0,255,0)

            # Create an empty image for each contour
            contour_img = image.copy()
            cv2.drawContours(contour_img, [cnt], 0, color, 5)

            # Resize contour image to fit into the square
            resized_img = cv2.resize(contour_img, (square_size, square_size))
            contour_images.append(resized_img)

    # Determine grid size
    num_contours = len(contour_images)
    grid_size = math.ceil(math.sqrt(num_contours))

    # Create a large image to hold the grid of contour images
    combined_image = np.zeros((grid_size * square_size, grid_size * square_size, 3), dtype=np.uint8)

    # Place each contour image in the grid
    for i, contour_img in enumerate(contour_images):
        row = i // grid_size
        col = i % grid_size
        start_y = row * square_size
        start_x = col * square_size
        combined_image[start_y:start_y + square_size, start_x:start_x + square_size] = contour_img

    return combined_image

# Example usage
cv2.imshow('Original Image', image)
cv2.imshow('Edge Map', edges)
processed_image = process_image('path_to_your_image.jpg')
cv2.imwrite(folder + 'edited.png', processed_image)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


True

: 

In [ ]:
import cv2

# Load the image
image_path = 'path_to_your_image.jpg'
image = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply edge detection or thresholding
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the original image
# -1 in the third argument to draw all contours
# (0, 255, 0) is the color (green in this case)
# 2 is the thickness of the contours
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

# Display the image with contours
cv2.imshow('Image with Contours', image)